In [13]:
import requests
import folium
import pandas as pd
from folium import IFrame
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# Fetch the data from the API for Vehicle
url_vehicle = 'http://127.0.0.1:9090/airquality/vehicle'
response_vehicle = requests.get(url_vehicle)
data_vehicle = response_vehicle.json()

# Convert JSON data to a DataFrame
df_vehicle = pd.json_normalize(data_vehicle)

# Function to create a pie chart and return the base64-encoded image
def create_pie_chart(row):
    labels = ['None', 'One', 'Two', 'Three', 'More than Four']
    sizes = [
        row['none_vehicle_per_dwelling'],
        row['one_vehicle_per_dwelling'],
        row['two_vehicle_per_dwelling'],
        row['three_vehicle_per_dwelling'],
        row['more_than_four_vehicle_per_dwelling']
    ]
    colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#c2c2f0']
    
    fig, ax = plt.subplots(figsize=(5, 5))  # Adjust the figure size
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    # Save the figure in a BytesIO object
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # Encode the image in base64
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    return img_base64

# Create a Folium map centered on a specific location
m = folium.Map(width=900, height=600, location=[-37.8014, 144.9958], zoom_start=10, tiles='OpenStreetMap')

# Define a function to create styled HTML for the popups
def create_popup_html(row):
    img_base64 = create_pie_chart(row)
    html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <h4>{row['site_name']}</h4>
        <p><strong>Health Advice:</strong> <span style="color: {row['health_advice_color']};">{row['health_advice']}</span></p>
        <p><strong>Value:</strong> {row['value']} {row['unit']}</p>
        <p><strong>Local Government Area:</strong> {row['lga_name']} ({row['lga_code']})</p>
        <p><strong>Vehicle Data per Dwelling:</strong></p>
        <img src="data:image/png;base64,{img_base64}" alt="Pie chart" width="250" height="250">
    </div>
    """
    return html

# Iterate over the DataFrame and add markers to the map
for index, row in df_vehicle.iterrows():
    popup_html = create_popup_html(row)
    iframe = IFrame(popup_html, width=350, height=400)
    popup = folium.Popup(iframe, max_width=350)
    
    folium.Marker(
        location=[row['site_location.lat'], row['site_location.lon']],
        popup=popup,
        tooltip=f"<strong>{row['site_name']}</strong>",
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(m)

# Display the map
m
